# Grab - Marchés publics

## Sommaire

## Environnement

In [7]:
import os
import sys
import time
import datetime
import re

import yaml

import lxml
import lxml.html

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


[Sommaire](Sommaire)

## Configuration

In [9]:
Yml_cfg = '''
Ebp: 
    Grab:
        keyword: Ixelles
        date_from: 01/01/2015
        date_to: 01/03/2015
        type_not_implemented: ''
        type_list :
            - 'Appel d''offres général'
            - 'Appel aux candidats'
            - 'Attribution'
            - 'Avis indicatif'
            - 'Erratum - Rectification'
            - 'Marché négocié'
            - 'Procédure négociée directe avec publicité'
    Send_to:
        - blabla
    Env:
        web_site_public: http://www.ebp.be
        web_site_private: 'https://abosl.ebp.be'
        web_page_login: /Main/Login
        web_page_logout: /Main/Logout
        login_key: coophelmet@gmail.com
        login_value: 86c40
'''

Cfg = None

[Sommaire](Sommaire)

## Log

In [13]:
def login(driver):
    global Cfg
    _str_page = '/Main/Login'
    driver.get("https://abosl.ebp.be" + _str_page)
    
    _cookies = driver.get_cookies()
    
    elm_username = driver.find_element_by_id("username")
    elm_username.send_keys(Cfg['Ebp']['Env']['login_key'])
    
    elm_password = driver.find_element_by_id("password")
    elm_password.send_keys(Cfg['Ebp']['Env']['login_value'])
    
    elm_remember = driver.find_element_by_id("remember")
    _b_is_checked = elm_remember.get_attribute("checked")
    if _b_is_checked != None or  _b_is_checked == 'true':
        elm_remember.click()
    _b_is_checked = elm_remember.get_attribute("checked")
    
    elm_login = driver.find_element_by_id("login")
    elm_login.submit()

    time.sleep(2)

def main_login():
    global Cfg
    global Driver

    Cfg = yaml.load(Yml_cfg)
    Driver = webdriver.Firefox()
    login(Driver)

main_login()

[Sommaire](Sommaire)

In [12]:
def logout(driver):
    _str_page = '/Main/Logout'
    driver.get("https://abosl.ebp.be" + _str_page)

logout(Driver)

[Sommaire](Sommaire)

## Search

Effectue la recherche déterminée par la configuration

In [14]:
def search(driver, str_keyword=None, str_date_from=None, str_date_to=None):
    global Cfg
    
    driver.get("https://abosl.ebp.be/Search")
    # Mots-clés
    _elm = driver.find_element_by_id("Keywords")
    if str_keyword != None:
        _elm.send_keys(str_keyword)
    else:
        _elm.send_keys(Cfg['Ebp']['Grab']['keyword'])
    # dans le texte complet 
    _lst_elm = driver.find_elements_by_name("PublicationTextPartId")
    for _elm in _lst_elm:
        _str_att = _elm.get_attribute('value')
        if _str_att == '1':
            _elm.click()
            break

    # Date de publication entre
    _elm = driver.find_element_by_id("DateFrom")
    _elm.clear();
    if str_date_from != None:
        _elm.send_keys(str_date_from)
    else:
        _elm.send_keys(Cfg['Ebp']['Grab']['date_from'])
    _elm = driver.find_element_by_id("DateTo")
    _elm.clear();
    if str_date_to != None:
        _elm.send_keys(str_date_to)
    else:
        _elm.send_keys(Cfg['Ebp']['Grab']['date_to'])

    # Rechercher dans
    _lst_elm = driver.find_elements_by_name("PublicationDatabase")
    for _elm in _lst_elm:
        _str_att = _elm.get_attribute('value')
        if _str_att == '2':
            _elm.click()
            break
   
    _elm = driver.find_element_by_id("search")
    _elm.submit()
    
    time.sleep(2)
    
    #print(driver.page_source)
    
search(Driver)

[Sommaire](Sommaire)

## Détails

Fenêtre détails

In [16]:
def get_detail(driver):
    # columns
    # . .   Source    Annonce    Date    Type    Description    Régions    Pays    Date limite
    
    _str_page =  driver.page_source
    _el_block_last = lxml.html.fromstring(_str_page)
    
    
    _str_type = "Appel d'offres général"
    _str_xpath_rows = '''.//table[@id='publication-list']//tr[td[position()=6 and contains(text(), "%(type)s")]]''' % {'type': _str_type}
    _str_xpath_a = '''.//a[@class='publication-relation-link']'''

    
    _str_page =  driver.page_source
    _el_page = lxml.html.fromstring(_str_page)
    _el_rows =_el_page.xpath(_str_xpath_rows)
    for _el_row in _el_rows
        print(lxml.etree.tostring(_el_rows[0], pretty_print=True))
        _el_as =_el_rows[0].xpath(_str_xpath_a
        for _el_a in _el_as:
            print(lxml.etree.tostring(_el_as[0], pretty_print=True))
    
get_detail(Driver)

b'<tr role="row" class="even">\n  <td class=" text-center dt-noborder">\n    <input type="checkbox" name="chkPublication" value="11615889"/>\n    <input type="hidden" value="0" class="dt-taskid"/>\n  </td>\n  <td class=" text-center dt-noborder">\n    <img data-isread="false" data-publicationid="11615889" class="icon read-icon" alt="" title="Non lu" data-toggle="tooltip" src="/Content/images/PubUnread.png"/>\n    <br/>\n    <img class="icon" alt="" title="Mot(s) cl&#233;(s) trouv&#233;(s)" data-toggle="tooltip" src="/Content/images/staron.png"/>\n    <br/>\n    <img data-content="Vous pouvez initier une demande, et nous estimons la probabilit&#233; qu\'un document soit disponible comme &#233;tant forte." data-html="true" data-original-title="" data-toggle="popover" rel="colorclip" alt="" src="/Content/images/DCE_N.png"/>\n    <br/>\n  </td>\n  <td class=" text-center"><img alt="" src="/Content/images/PublicationBulletinTypeId001.png"/><br/>BdA 52</td>\n  <td class=" text-right">505045<

In [ ]:
[Sommaire](Sommaire)

## Authentification

Cookies ?

In [5]:
import requests

Str_login_key = 'coophelmet@gmail.com'
Str_login_value = '86c40'
    
Str_web_site = 'www.ebp.be'
Str_web_page_login =  'https://abosl.ebp.be/Main/Login'
    
_rqs = requests.get(Str_web_page_login) # auth=(Str_login_key, Str_login_value))
_rqs_status_code = _rqs.status_code
print('_rqs_status_code', _rqs_status_code)
_rqs_headers_content_type = _rqs.headers['content-type']
print('_rqs_headers_content_type', _rqs_headers_content_type)
_rqs_encoding = _rqs.encoding
print('_rqs_encoding', _rqs_encoding)
_rqs_text = _rqs.text
print('_rqs_text', _rqs_text)
#_rqs_json = _rqs.json()
#print(_rqs_json)
_rqs_cookies = _rqs.cookies
print('_rqs_cookies', _rqs_cookies)

_str_form_action = '/Main/Login"'
# gIppCVA1dqBepQjk--26wbTBsngHaTSRiiNZgX5kjnrmcBggiS6tIbN9Jq9J4eclOrbXJ3LduvEJEhwTDQx6vFE8Ljw1
_str_form__RequestVerificationToken = ''
_str_form_name_id_username = Str_login_key
_str_form_name_id_password = Str_login_value
_str_form_name_id_remember = 'on'
_str_form_submit_id = 'login'

# SnJFe02jaZCKKXFmXkIVeT0IZIsneT8tEnRHFHHoU6lIfQZ5KDd1ExoWSC-ZkMXl_q7_BuPFMP6J_JLd7SyhfneJ6LA1
str_cookie__RequestVerificationToken = ''

_rqs_status_code 200
_rqs_headers_content_type text/html; charset=utf-8
_rqs_encoding utf-8
_rqs_text 
<!DOCTYPE html>
<!--[if lt IE 9]><html class="ie" lang="en-US"><![endif]-->
<!--[if gt IE 9]><!-->
<html lang="en-US">
<!--<![endif]-->
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>EBP | Login</title>
    <link rel="shortcut icon" type="image/x-icon" href="/Content/images/FaviconDiv001.ico" />
    <link href="/common?v=a1hgRnpzGZqkiJby_cbarqHEKgrDlUEXEWrUZ1aBS7w1" rel="stylesheet"/>


        <link rel="stylesheet" href="/Content/less/ebp.min.css?v=2.8.4" />
    <!--[if lt IE 9]><script type="text/javascript" src="/Scripts/respond.min.js"></script><![endif]-->
    <script src="//ajax.googleapis.com/ajax/libs/jquery/1.11.2/jquery.min.js"></script>
    <script src="/Scripts/bootstrap.min.js"></script>
    <script src="/Scripts/jquery.dataTables-1.10.4.min.js"></scrip

## Recherche

Page = 

## Sélection

In [ ]:
qsdf

## Téléchargement

In [ ]:
qsdf